In [ ]:
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.4
    Uninstalling spacy-3.5.4:
      Successfully uninstalled spacy-3.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.6.0 which is incompatible.


In [ ]:
!python -m spacy download es_core_news_sm

2023-07-20 04:24:42.623713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 04:24:44.066283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

In [ ]:
DataBase = {
        "tabla": "productos",
        "columnas": ["id", "codigo", "cantidad", "descripcion"]
}

Operator = {
    "!=": ["distinto", "diferente","diverso","desigual","desemejante"],
    "<": ["menos", "anterior", "antes", "menores","pequeños"],
    ">": ["mayor", "posterior", "despues", "mayores","luego"],
    ">=": ["mayor igual", "igual mayor","igual y posterior", "igual y despues", "igual y mayores","luego"],
    "<=": ["menor igual", "igual menor","igual y anterior", "igual y antes", "igual y menores","igual y pequeños"],
    "not": ["no"]
}

In [ ]:
action=  {
        "SELECT": ["dame","seleccionar", "obtener", "extraer", "mostrar"],
}

In [ ]:
def getWords(frase):
    doc = nlp(frase)
    palabras_importantes = []

    for token in doc:
        if token.pos_ in ["NOUN", "PROPN", "VERB", "ADJ", "ADV", "NUM"] or token.ent_type_ == "DATE":
            palabras_importantes.append(token.text)

    palabrasImportantes = ' '.join(palabras_importantes)

    return palabrasImportantes

In [ ]:
def getAction(frase_ingresada, matcher, action):
  doc = nlp(frase_ingresada.lower())
  for accion, palabras in action.items():
      patterns = [[{"LOWER": palabra}] for palabra in palabras]
      matcher.add(accion, patterns)

  matches = matcher(doc)

  for match_id, start, end in matches:
      accion = nlp.vocab.strings[match_id]
      frase_ingresada = " ".join([token.text for token in doc if token.i < start or token.i >= end])

      return accion, frase_ingresada.strip()
  return None,frase_ingresada

In [ ]:
def getSimilarity(frase, DataBase):
    doc = nlp(frase)
    tablasNew = set()
    columnasNew = set()

    for token_frase in doc:
        similitud_tabla = token_frase.similarity(nlp(DataBase["tabla"]))
        if similitud_tabla >= 0.6:
            tablasNew.add(DataBase["tabla"])
        for token_frase in doc:
            for columna in DataBase["columnas"]:
                similitud = token_frase.similarity(nlp(columna))
                if similitud > 0.6 and columna in token_frase.text:
                    columnasNew.add(columna)
    return tablasNew, columnasNew

In [ ]:
def getOperator(frase, matcher, Operator):
    doc = nlp(frase.lower())

    for operador, palabras in Operator.items():
        patterns = [[{"LOWER": palabra}] for palabra in palabras]
        matcher.add(operador.lower(), patterns)

    matches = matcher(doc)

    for match_id, start, end in matches:
        operador = nlp.vocab.strings[match_id]
        frase = " ".join([token.text for token in doc if token.i < start or token.i >= end])

        return operador, frase.strip()

    return None, frase

In [ ]:
def extraWords(frase, matcher, DataBase,action,Operator):
    p_importantes = getWords(frase)
    accion, fraseDevuelta = getAction(p_importantes, matcher, action)
    resultados_tabla, resultados_columnas = getSimilarity(frase, DataBase)

    p_restantes = set(fraseDevuelta.split()) - resultados_tabla - resultados_columnas
    p_res = ' '.join(p_restantes)

    operador, frase_2 = getOperator(p_res, matcher, Operator)

    condicion = ""
    if operador and frase_2:
        doc = nlp(frase_2)
        for token in doc:
            if token.pos_ in [ "PROPN","NUM"] or token.ent_type_ == "DATE":
                condicion += token.text + " "
            elif token.pos_ in ["VERB", "ADJ"]:
                condicion += token.lemma_ + " "

    return accion, resultados_tabla, resultados_columnas, operador,frase_2, condicion.strip()

In [ ]:
def consultSql(accion, tablas, columnas, operador, condicion):
    consulta = ""
    condiciones = []
    if columnas:
      colum_sql_list = list(columnas)
      if operador:
        operaciones_list = list(operador)
        condiciones = f"{colum_sql_list[0]} {operaciones_list[0]} {condicion}"
    if accion == "SELECT":
        if columnas:
            consulta += f"SELECT {', '.join(columnas)} FROM {', '.join(tablas)}"
        else:
            consulta += f"SELECT * FROM {', '.join(tablas)}"

        if condiciones:
            consulta += f" WHERE {condiciones}"
    return consulta

In [ ]:
sentence = "obtener fecah de nacimiento de los estudiantes"

accion,tableSql, columnSql, operator, rest, condition = extraWords(sentence, matcher, DataBase,action, Operator)
sqlQuery = consultSql(accion,tableSql, columnSql, operator, condition)


<ipython-input-9-b52548da82f2>:7: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similitud_tabla = token_frase.similarity(nlp(DataBase["tabla"]))
<ipython-input-9-b52548da82f2>:12: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vecto

In [ ]:
print("SQL Query:",sqlQuery)

SQL Query: SELECT * FROM 
